# Building Product Recommendation Engine with Amazon Personalize

In this lab, we would like to use Amazon.com’s customer rating data to build product recommendation plugin for our website. We will use Amazon Personalize to train the recommender model and to host the recommendation inference. In addition, we will test out the inference and display the items that user rated and items that are recommended for that user.

## Download and prepare sample dataset

In [1]:
!curl -o ./metadata.json.gz http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_AMAZON_FASHION.json.gz
!curl -o ./ratings.json.gz http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/AMAZON_FASHION.json.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 40.7M  100 40.7M    0     0  6432k      0  0:00:06  0:00:06 --:--:-- 8363k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 88.8M  100 88.8M    0     0  7738k      0  0:00:11  0:00:11 --:--:-- 8693k


Data by:    
Justifying recommendations using distantly-labeled reviews and fined-grained aspects. Jianmo Ni, Jiacheng Li, Julian McAuley. Empirical Methods in Natural Language Processing (EMNLP), 2019.

In [2]:
!gunzip -f ratings.json.gz
!gunzip -f metadata.json.gz

**Loading ratings into panda data frame**

In [3]:
import pandas as pd 
ratings_df = pd.read_json("./ratings.json", lines=True)

In [4]:
ratings_df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5,True,"10 20, 2014",A1D4G1SNUZWQOT,7106116521,Tracy,Exactly what I needed.,perfect replacements!!,1413763200,NaN,NaN,NaN
1,2,True,"09 28, 2014",A3DDWDH9PX2YX2,7106116521,Sonja Lau,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400,3.0,NaN,NaN
2,4,False,"08 25, 2014",A2MWC41EW7XL15,7106116521,Kathleen,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800,NaN,NaN,NaN
3,2,True,"08 24, 2014",A2UH2QQ275NV45,7106116521,Jodi Stoner,too tiny an opening,Two Stars,1408838400,NaN,NaN,NaN
4,3,False,"07 27, 2014",A89F3LQADZBS5,7106116521,Alexander D.,Okay,Three Stars,1406419200,NaN,NaN,NaN


**Include only important columns**

In [5]:
ratings_df = ratings_df[["reviewerID","asin","overall","unixReviewTime"]]

In [6]:
ratings_df.head()

,reviewerID,asin,overall,unixReviewTime
0,A1D4G1SNUZWQOT,7106116521,5,1413763200
1,A3DDWDH9PX2YX2,7106116521,2,1411862400
2,A2MWC41EW7XL15,7106116521,4,1408924800
3,A2UH2QQ275NV45,7106116521,2,1408838400
4,A89F3LQADZBS5,7106116521,3,1406419200


**This is how the ratings file look like (first 5 lines)**

In [7]:
import boto3

personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [8]:
ratings_df = ratings_df.rename(columns={'reviewerID':'USER_ID', 'asin':'ITEM_ID', 'unixReviewTime':'TIMESTAMP', 'overall':'EVENT_VALUE'})

In [9]:
ratings_df['EVENT_TYPE'] = "reviewed"
ratings_df['EVENT_VALUE'] = ratings_df['EVENT_VALUE'].astype(float)

In [10]:
ratings_df.head()

,USER_ID,ITEM_ID,EVENT_VALUE,TIMESTAMP,EVENT_TYPE
0,A1D4G1SNUZWQOT,7106116521,5.0,1413763200,reviewed
1,A3DDWDH9PX2YX2,7106116521,2.0,1411862400,reviewed
2,A2MWC41EW7XL15,7106116521,4.0,1408924800,reviewed
3,A2UH2QQ275NV45,7106116521,2.0,1408838400,reviewed
4,A89F3LQADZBS5,7106116521,3.0,1406419200,reviewed


## Specify a bucket and data output location

In [11]:
import sagemaker
sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = 'recommendation-engine-with-personalize-fashion'
filename = "clean_product_ratings.csv"

**Filter only users' ratings that are > 3, and reformat ratings file for Amazon Personalize input**

In [12]:
ratings_df.to_csv(filename, index=False)

boto3.Session().resource('s3').Bucket(bucket).Object("{}/{}".format(prefix,filename)).upload_file(filename)



## Ingest data to Amazon Personalize

**Specify naming and threshold**

In [13]:
base_name = "fashion"
iteration = "2"
review_star_threshold = 4.0

**Create user-interactions schema**

In [14]:
import json
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
        },
        {
            "name": "EVENT_VALUE",
            "type": "float"
        },
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "{}-{}-interaction-schema".format(base_name, iteration),
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))


{
  "schemaArn": "arn:aws:personalize:ap-southeast-1:344028372807:schema/fashion-2-interaction-schema",
  "ResponseMetadata": {
    "RequestId": "fff2ed2a-ddb7-49e7-b182-e9ab9629feee",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 29 Oct 2020 01:52:01 GMT",
      "x-amzn-requestid": "fff2ed2a-ddb7-49e7-b182-e9ab9629feee",
      "content-length": "99",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


**Create Dataset Group**

In [15]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "{}-{}-dataset-group".format(base_name, iteration)
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:ap-southeast-1:344028372807:dataset-group/fashion-2-dataset-group",
  "ResponseMetadata": {
    "RequestId": "e367802c-0a42-4426-88e6-e99af1d27f6d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 29 Oct 2020 01:52:00 GMT",
      "x-amzn-requestid": "e367802c-0a42-4426-88e6-e99af1d27f6d",
      "content-length": "107",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [16]:
import time
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


**Create dataset**

In [17]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "{}-{}-interaction-dataset".format(base_name, iteration),
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:ap-southeast-1:344028372807:dataset/fashion-2-dataset-group/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "1ac9aba8-9554-4534-bce9-80d8f22b89f0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 29 Oct 2020 01:53:01 GMT",
      "x-amzn-requestid": "1ac9aba8-9554-4534-bce9-80d8f22b89f0",
      "content-length": "109",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


**Attach Policy to S3 Bucket**

In [18]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy{}{}".format(base_name, iteration),
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': 'C07F050A06074755',
  'HostId': 'pkTXDiJ4BXx0+foShS7lr1a9Y5bIryR87Iwu0BCTfZCowIME0fiaxvoFU1BKF0q+D9Bq9gQSF5Q=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'pkTXDiJ4BXx0+foShS7lr1a9Y5bIryR87Iwu0BCTfZCowIME0fiaxvoFU1BKF0q+D9Bq9gQSF5Q=',
   'x-amz-request-id': 'C07F050A06074755',
   'date': 'Thu, 29 Oct 2020 01:53:02 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

**Create Personalize Role**

In [19]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleForRecommendation{}{}".format(base_name, iteration)
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::344028372807:role/PersonalizeRoleForRecommendationfashion2


In [20]:
import json
role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Action": "s3:*",
          "Resource":"*"
        }
    ]
}
response = iam.create_policy(
    PolicyName='PersonalizeToS3{}{}'.format(base_name, iteration),
    PolicyDocument=json.dumps(role_policy_document)
)

response = iam.attach_role_policy(
    RoleName=role_name, PolicyArn=response['Policy']['Arn'])

time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

**Create dataset import job**

In [21]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "{}-{}-interaction-import-job".format(base_name, iteration),
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}/{}".format(bucket, prefix, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:ap-southeast-1:344028372807:dataset-import-job/fashion-2-interaction-import-job",
  "ResponseMetadata": {
    "RequestId": "098d2e58-0930-48de-8b2c-cbcd8010f8ca",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 29 Oct 2020 01:55:05 GMT",
      "x-amzn-requestid": "098d2e58-0930-48de-8b2c-cbcd8010f8ca",
      "content-length": "125",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


**Wait for Dataset Import Job to Have ACTIVE Status**

In [22]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)



DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


## Train

**Select Recipe**

In [23]:
list_recipes_response = personalize.list_recipes()
recipe_arn_up = "arn:aws:personalize:::recipe/aws-user-personalization"
recipe_arn_hrnn = "arn:aws:personalize:::recipe/aws-hrnn"

**Define Solution Configuration**

In [80]:

solution_config = {
        "eventValueThreshold": str(review_star_threshold),
        "featureTransformationParameters": {
            "min_user_history_length_percentile" : "0.0", # increase to remove popular item
            "max_user_history_length_percentile" : "0.99", # stay
        },
        "algorithmHyperParameters": {
            "hidden_dimension" : "159"
        },
        "hpoConfig": {
            "algorithmHyperParameterRanges": {
                'integerHyperParameterRanges': [
                    {
                        'name': 'bptt', # decrease to discard long-term factor that results to purchases
                        'minValue': 3,
                        'maxValue': 10
                    },
                ],
                "continuousHyperParameterRanges": [],
                'categoricalHyperParameterRanges': [
                    #{
                    #    'name': 'recency_mask', # set to false for discarding recency factor of purchases
                    #    'values': ['true','false']
                    #},
                ],
            },
            "hpoResourceConfig": {
                "maxNumberOfTrainingJobs": "40",
                "maxParallelTrainingJobs": "10"
            }
        }
    }

**Create Solution for User Personalization**

In [81]:
create_solution_response_up = personalize.create_solution(
    name = "{}-{}-up-solution2".format(base_name, iteration),
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn_up,
    eventType = 'reviewed',
    performHPO = True,
    solutionConfig = solution_config
)

create_solution_response_hrnn = personalize.create_solution(
    name = "{}-{}-hrnn-solution2".format(base_name, iteration),
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn_hrnn,
    eventType = 'reviewed',
    performHPO = True,
    solutionConfig = solution_config
)

solution_arn_up = create_solution_response_up['solutionArn']
solution_arn_hrnn = create_solution_response_hrnn['solutionArn']
print(json.dumps(create_solution_response_up, indent=2))
print(json.dumps(create_solution_response_hrnn, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-southeast-1:344028372807:solution/fashion-2-up-solution2",
  "ResponseMetadata": {
    "RequestId": "a27e4d29-6164-4282-b9dd-0b12481159c9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 11 Nov 2020 09:35:25 GMT",
      "x-amzn-requestid": "a27e4d29-6164-4282-b9dd-0b12481159c9",
      "content-length": "97",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}
{
  "solutionArn": "arn:aws:personalize:ap-southeast-1:344028372807:solution/fashion-2-hrnn-solution2",
  "ResponseMetadata": {
    "RequestId": "0910aede-4ff0-4be1-b04e-46e161daebca",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 11 Nov 2020 09:35:24 GMT",
      "x-amzn-requestid": "0910aede-4ff0-4be1-b04e-46e161daebca",
      "content-length": "99",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


**Create Solution Version**

In [82]:
create_solution_version_response_up = personalize.create_solution_version(
    solutionArn = solution_arn_up
)

create_solution_version_response_hrnn = personalize.create_solution_version(
    solutionArn = solution_arn_hrnn
)

solution_version_arn_up = create_solution_version_response_up['solutionVersionArn']
solution_version_arn_hrnn = create_solution_version_response_hrnn['solutionVersionArn']
print(json.dumps(create_solution_version_response_up, indent=2))
print(json.dumps(create_solution_version_response_hrnn, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-southeast-1:344028372807:solution/fashion-2-up-solution2/dad9debf",
  "ResponseMetadata": {
    "RequestId": "78c19ec9-2740-4232-85bd-48c05cfe4428",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 11 Nov 2020 09:35:27 GMT",
      "x-amzn-requestid": "78c19ec9-2740-4232-85bd-48c05cfe4428",
      "content-length": "113",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}
{
  "solutionVersionArn": "arn:aws:personalize:ap-southeast-1:344028372807:solution/fashion-2-hrnn-solution2/69e5ec97",
  "ResponseMetadata": {
    "RequestId": "00430f2e-4016-46c3-9d85-f1f2b38482d8",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Wed, 11 Nov 2020 09:35:28 GMT",
      "x-amzn-requestid": "00430f2e-4016-46c3-9d85-f1f2b38482d8",
      "content-length": "115",
      "connection": "keep-alive"
    },
   

**Wait for Solution Version to Have ACTIVE Status**

In [71]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn_up
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS


KeyboardInterrupt: 

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn_hrnn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

**Get Metrics of Solution**

In [ ]:
get_solution_metrics_response_up = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn_up
)
get_solution_metrics_response_hrnn = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn_hrnn
)

print(json.dumps(get_solution_metrics_response_up, indent=2))
print(json.dumps(get_solution_metrics_response_hrnn, indent=2))

## Deploy

**Create Campaign**

In [ ]:
create_campaign_response_up = personalize.create_campaign(
    name = "{}-{}-campaign-up".format(base_name, iteration),
    solutionVersionArn = solution_version_arn_up,
    minProvisionedTPS = 1
)
create_campaign_response_hrnn = personalize.create_campaign(
    name = "{}-{}-campaign-hrnn".format(base_name, iteration),
    solutionVersionArn = solution_version_arn_hrnn,
    minProvisionedTPS = 1
)

campaign_arn_up = create_campaign_response_up['campaignArn']
campaign_arn_hrnn = create_campaign_response_hrnn['campaignArn']
print(json.dumps(create_campaign_response_up, indent=2))
print(json.dumps(create_campaign_response_hrnn, indent=2))


**Wait for Campaign to Have ACTIVE Status**

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn_up
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn_hrnn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Clean and upload product metadata

In [ ]:
filename = 'metadata.json'
boto3.Session().resource('s3').Bucket(bucket).Object("{}/{}".format(prefix,filename)).upload_file(filename)

## Prepare for inference

**Prepare method to enrich the items information with title and image URL**

In [ ]:
import json
def encrich_with_metadata(products):
    client = boto3.client('s3')
    r = client.select_object_content(
        Bucket=bucket,
        Key="{}/metadata.json".format(prefix),
        Expression="SELECT s.image, s.asin, s.title FROM S3Object s WHERE s.asin IN {}".format(products),
        #Expression="SELECT s.imUrl, s.asin, s.title FROM S3Object s WHERE s.asin IN {}".format(products),
        ExpressionType='SQL',
        RequestProgress={
            'Enabled': False
        },
        InputSerialization={
            'JSON': {
                'Type': 'LINES'
            }
        },
        OutputSerialization={
            'JSON':{
                'RecordDelimiter': '\n',
            }
        },
    )
    output = []
    for event in r['Payload']:
        if 'Records' in event:
            recs = event['Records']['Payload'].decode('utf-8').strip().split("\n")
            recs = list(map(lambda x: json.loads(x), recs))
            output += recs
    return output

**Build filter to exlude items that user has purchased and reviewed**

In [ ]:
filter_response = personalize.create_filter(
    name='{}-{}-exclude-purchases'.format(base_name, iteration),
    datasetGroupArn= dataset_group_arn,
    filterExpression= 'EXCLUDE ItemID WHERE Interactions.event_type IN ("reviewed")'
)

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_filter_response = personalize.describe_filter(
        filterArn = filter_response['filterArn']
    )
    status = describe_filter_response["filter"]["status"]
    print("Filter: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

**Get a sample user to test**

In [ ]:
# Get a user who has considerable number of reviews
#user_id = ratings_df[ratings_df['EVENT_VALUE'] >= review_star_threshold]['USER_ID'].value_counts()[:2000].index.to_list()[1995]
user_id = ratings_df[ratings_df['EVENT_VALUE'] >= review_star_threshold]['USER_ID'].value_counts()[:2000].index.to_list()[20]

**Define method to display items for view purpose**

In [ ]:
import re
def display_items(items):
    image_string = ""
    i = 1
    for item in items:
        if 'score' in item:
            caption = "{}---Score:{}---Name: {}---ASIN:{}".format(str(i),item['score'], item['title'],item['asin'])
        else:
            caption = "{}---Name: {}---ASIN:{}".format(str(i),item['title'],item['asin'])
        if len(item['image']) > 0:
            image = item['image'][0]
            image = re.sub(r'SR..,..','SR200,200',image)
            image = re.sub(r'US..','US200',image)
            image = re.sub(r'SS..','SS200',image)
            image = re.sub(r'SX..','SX200',image)
            image = re.sub(r'SY..','SX200',image)
            image = re.sub(r'CR,0,0,..,..','CR,0,0,200,200',image)
            image_string += '<figure style="float:left;"><img src="{}" alt="" width="1"/><figcaption ><center>{}</center></figcaption></figure></br>'.format(image,caption)
        else:
            image_string += '<figure style="float:left;"><img src="" alt="" width="1"/><figcaption ><center>{}</center></figcaption></figure></br>'.format(caption)
        i = i+1
    return image_string

**Get actual items that user reviewed with rating > 3 and enrich with title and image URL**

In [ ]:
actual_item_list = list(ratings_df[(ratings_df["USER_ID"] == user_id) & (ratings_df['EVENT_VALUE'] >= review_star_threshold)]['ITEM_ID'])
actual_items = encrich_with_metadata(actual_item_list)

In [ ]:
from IPython.display import HTML
HTML(data=display_items(actual_items))

**Get recommendation for user personalize recipe and enrich recommended items with title and images**


In [ ]:
get_recommendations_response_up = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn_up,
    userId = str(user_id),
    numResults=10,
    filterArn=filter_response['filterArn']
)

recommended_item_list = list(map(lambda x: x['itemId'], get_recommendations_response_up['itemList']))
print(get_recommendations_response_up['itemList'])

    

**Prepare for display**

In [ ]:
recommended_items = encrich_with_metadata(recommended_item_list)
items_dictionary = {}
for item in get_recommendations_response_up['itemList']:
    items_dictionary[item['itemId']]=item['score']                                           
for item in recommended_items:
    item['score'] = items_dictionary[item['asin']]
recommended_items.sort(key=lambda x: x['score'], reverse=True)

**These are the actual items that user reviewed**

In [ ]:
from IPython.display import HTML
HTML(data=display_items(recommended_items))

**Get recommendation for user personalize recipe and enrich recommended items with title and images**


In [ ]:
get_recommendations_response_hrnn = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn_hrnn,
    userId = str(user_id),
    numResults=10,
    filterArn=filter_response['filterArn']
)

recommended_item_list = list(map(lambda x: x['itemId'], get_recommendations_response_hrnn['itemList']))
print(get_recommendations_response_hrnn['itemList'])

    

**Prepare for display**

In [ ]:
recommended_items = encrich_with_metadata(recommended_item_list)
items_dictionary = {}
for item in get_recommendations_response_hrnn['itemList']:
    items_dictionary[item['itemId']]=item['score']                                           
for item in recommended_items:
    item['score'] = items_dictionary[item['asin']]
recommended_items.sort(key=lambda x: x['score'], reverse=True)

**These are the actual items that user reviewed**

In [ ]:
from IPython.display import HTML
HTML(data=display_items(recommended_items))